In [19]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction to CS203

## What a "real" program look like?

In [20]:
! gcc hello_world.c -o hello_world
! ./hello_world
! objdump -D hello_world

Hello, world!
Where is the string? 0x556007527010

hello_world:     file format elf64-x86-64


Disassembly of section .interp:

0000000000000318 <.interp>:
 318:	2f                   	(bad)  
 319:	6c                   	insb   (%dx),%es:(%rdi)
 31a:	69 62 36 34 2f 6c 64 	imul   $0x646c2f34,0x36(%rdx),%esp
 321:	2d 6c 69 6e 75       	sub    $0x756e696c,%eax
 326:	78 2d                	js     355 <_init-0xcab>
 328:	78 38                	js     362 <_init-0xc9e>
 32a:	36 2d 36 34 2e 73    	ss sub $0x732e3436,%eax
 330:	6f                   	outsl  %ds:(%rsi),(%dx)
 331:	2e 32 00             	xor    %cs:(%rax),%al

Disassembly of section .note.gnu.property:

0000000000000338 <.note.gnu.property>:
 338:	04 00                	add    $0x0,%al
 33a:	00 00                	add    %al,(%rax)
 33c:	10 00                	adc    %al,(%rax)
 33e:	00 00                	add    %al,(%rax)
 340:	05 00 00 00 47       	add    $0x47000000,%eax
 345:	4e 55                	rex.WRX push %rbp
 347:	00 02      

In [21]:
render_code("hello_world.c")

// hello_world.c:1-10 (10 lines)
#include <stdio.h>

char hello_string[] = "Hello, world!\n";

int main()
{
    printf("%s",hello_string);
    printf("Where is the string? %p\n", hello_string);
    return 0;
}

## To sort or not to sort?

Consider the following code snippet, what the performance looks like when option is set to 1 or 0?

In [33]:
render_code("calculate_sum.c")

// calculate_sum.c:1-15 (15 lines)
#ifdef __cplusplus
extern "C" {
#endif
long long calculate_sum(int *data, unsigned int size, int threshold)
{
    long long sum=0;
    for (unsigned i = 0; i < size; ++i) {
        if (data[i] >= threshold)
                sum ++;
    }
    return sum;
}
#ifdef __cplusplus
}
#endif

In [34]:
render_code("arraySort.cpp")

// arraySort.cpp:1-37 (37 lines)
#include <algorithm>
#include <ctime>
#include <iostream>
#include <climits>

#ifdef __cplusplus
extern "C" {
#endif
long long calculate_sum(int *data, unsigned int size, int threshold);
#ifdef __cplusplus
}
#endif


int main(int argc, char **argv)
{
    // generate data
//    const unsigned arraySize = 262144;
    int arraySize = atoi(argv[1]);
    int *data = new int[arraySize];
    long long sum = 0;

    int iterations = atoi(argv[2]);
    int option = atoi(argv[3]);

    for (unsigned c = 0; c < arraySize; ++c)
        data[c] = std::rand();

    if(option)
        std::sort(data, data + arraySize);

    for (unsigned i = 0; i < iterations; ++i) {
        sum += calculate_sum(data, arraySize, std::rand());
    }
    std::cout << "sum = " << sum << std::endl;
}

In [ ]:
! make clean; make

In [ ]:
! time ./arraySort 262144 1000 1

In [ ]:
! time ./arraySort 262144 1000 0

## Matrix Multiplications

Matrix multiplications in general has an O($n^3$) complexity, how does the complexity work on real computers?

In [ ]:
render_code("mm.c", show=["//START","//END"])

In [35]:
! rm mm; make mm
! time ./mm 1024

rm: cannot remove 'mm': No such file or directory
gcc -O3 mm.c perfstats.o -o mm 
2.121229 seconds

real	0m2.134s
user	0m2.128s
sys	0m0.004s


In [36]:
! time ./mm 512

0.151736 seconds

real	0m0.155s
user	0m0.155s
sys	0m0.000s


In [37]:
! time ./mm 2048

24.661503 seconds

real	0m24.692s
user	0m24.666s
sys	0m0.024s


In [38]:
! time ./mm 4096

308.039017 seconds

real	5m8.164s
user	5m8.051s
sys	0m0.104s


## Is bitwise worth doing?

In [40]:
render_code("instructions/arithmetics_2.c")

// instructions/arithmetics_2.c:1-29 (29 lines)
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

int main()
{
    int i,sum=0,data[131072],previous=0,tmp=0, tmp_1=0;
    struct timeval time_start, time_end, program_start, program_end;
    for(i=0;i<131072;i++)
        data[i]=rand()%8738;
#ifdef A
    gettimeofday(&time_start, NULL);
    for(i=0;i<1000000000;i++)
    {
        sum+=data[(i*13) & 131071];
    }
    gettimeofday(&time_end, NULL);
    printf("sum: %d %lf seconds\n",sum, ((time_end.tv_sec * 1000000 + time_end.tv_usec) - (time_start.tv_sec * 1000000 + time_start.tv_usec))/1000000.0);
#else
    gettimeofday(&time_start, NULL);
    for(i=0;i<1000000000;i++)
    {
        sum+=data[((i << 4) - (i << 2) + i) & 131071];
    }
    gettimeofday(&time_end, NULL);
    printf("sum: %d %lf seconds\n",sum,((time_end.tv_sec * 1000000 + time_end.tv_usec) - (time_start.tv_sec * 1000000 + time_start.tv_usec))/1000000.0);
#endif        
    return 0;
}

In [39]:
! make clean; make -C instructions
! time ./instructions/arithmetics_2_A
! time ./instructions/arithmetics_2_B

rm -f madd arraySort bitonic quicksort mm
make: Entering directory '/nfshome/htseng/courses/CS203/demo/introduction/instructions'
make: Nothing to be done for 'all'.
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/introduction/instructions'
sum: 1561731795 1.398125 seconds

real	0m1.400s
user	0m1.399s
sys	0m0.000s
sum: 1561731795 1.380732 seconds

real	0m1.385s
user	0m1.384s
sys	0m0.000s


## Algorithm on GPUs?
Consider quicksort and bitsort, which performs better on GPUs?

### Quick sort

In [ ]:
! nvidia-smi -a

In [ ]:
render_code("quicksort.cu", show="quickSortIterative")

In [ ]:
render_code("quicksort.cu", show="partition")

### Bitonic sort

In [ ]:
render_code("bitonic.cu", show="bitonic_sort")

In [ ]:
render_code("bitonic.cu", show="bitonic_sort_step")

In [ ]:
! ./bitonic

In [ ]:
!./quicksort

In [ ]:
! make cpuSort.cpp
!time ./arraySort 67108864 1 1